# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

In [4]:
import pandas as pd
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [2]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [3]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

0    Евгения гр.ПМ19-1
1         Илья пм 20-4
2            Анна 20-3
dtype: object

3. Разбейте текст формулировки задачи 2 на слова.

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны.

In [10]:
data = pd.read_csv("06_string_data/recipes_sample.csv", sep=',')
data_ = data.loc[:, ['id', 'minutes']]
print("|{: ^14}|{: ^11}|".format(*data_.columns))
print("|" + "-" * 26 + "|")
for row in data_.sample(5).values:
    print(f"|{row[0]: ^14}|{row[1]: ^11}|")

|      id      |  minutes  |
|--------------------------|
|    436816    |    25     |
|    337244    |    20     |
|    77043     |    30     |
|    112057    |    110    |
|    201780    |    20     |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [ ]:
assert (
        show_info(
            name="george s at the cove black bean soup",
            steps=[
                "clean the leeks and discard the dark green portions",
                "cut the leeks lengthwise then into one-inch pieces",
                "melt the butter in a medium skillet , med",
            ],
            minutes=90,
            author_id=35193,
        )
        == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

In [32]:
import xml

with open('06_string_data/steps_sample.xml', 'r') as f:
    steps_sample_2 = BeautifulSoup(f, features='xml')


def show_info(id_: int, recipes_sample: pd.DataFrame, steps_sample: BeautifulSoup):
    recipes = recipes_sample[recipes_sample['id'] == id_]
    recipes = recipes.iloc[0]
    steps_id = steps_sample.find_all('id')
    steps = None
    print(f"\"{recipes['name']}\"\n")
    for ID in steps_id:
        if int(ID.text) == id_:
            steps = ID.parent.find('steps').find_all('step')

    for index, step in enumerate(steps):
        print(f"{index}. {step.text}")
    print('--------')
    print(f"Автор: {recipes['contributor_id']}\nСреднее время приготовления: {recipes['minutes']} минут")
    # s = '"{}"\n\n'.format(list_[0][list_[0]["id"] == id_]["name"].item())


show_info(170895, data, steps_sample_2)

"leeks and parsnips  sauteed or creamed"

170895
0. clean the leeks and discard the dark green portions
1. cut the leeks lengthwise then into one-inch pieces
2. melt the butter in a medium skillet , med
3. heat
4. add the garlic and fry 'til fragrant
5. add leeks and fry until the leeks are tender , about 6-minutes
6. meanwhile , peel and chunk the parsnips into one-inch pieces
7. place in a steaming basket and steam 'til they are as tender as you prefer
8. i like them fork-tender
9. drain parsnips and add to the skillet with the leeks
10. add salt and pepper
11. gently sautee together for 5-minutes
12. at this point you can serve it , or continue on and cream it:
13. in a jar with a screw top , add the half-n-half and arrowroot
14. shake 'til blended
15. turn heat to low under the leeks and parsnips
16. pour in the arrowroot mixture , stirring gently as you pour
17. if too thick , gradually add the water
18. let simmer for a couple of minutes
19. taste to adjust seasoning , probably a

: ## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [39]:
import re
id_ = 25082
with open('06_string_data/steps_sample.xml', 'r') as f:
    steps_sample = BeautifulSoup(f, features='xml')
steps_id = steps_sample.find_all('id')
for ID in steps_id:
    if int(ID.text) == id_:
        steps = ID.parent.find('steps').find_all('step')

for index, step in enumerate(steps):
    r = re.search('\d+ (hours|hour|minute|minutes)', step.text)
    if r:
        print(r.groups())

('minute',)
('minute',)
('hours',)
('minute',)
('minute',)


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.
